In [1]:
import time
import dask.dataframe as dd
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
print("read_csv")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuCharDD = dd.read_csv(
    "chartevents.csv",
    dtype={
        "caregiver_id": "float64",
        "value": "object",
        "valuenum": "float64",
        "warning": "float64",
    },
)

print("end read_csv")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

admiDD = dd.read_csv(
    "20230525_160146mimiciv_hosp_admi.csv",
    dtype={'deathtime': 'object'}
)


print("end read_csv")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

read_csv
20230530_155352
end read_csv
20230530_155352
end read_csv
20230530_155352


In [3]:
admiDD.dtypes

subject_id               int64
hadm_id                  int64
admittime               object
dischtime               object
deathtime               object
race                    object
hospital_expire_flag     int64
dtype: object

In [4]:
icuCharDD.dtypes

subject_id        int64
hadm_id           int64
stay_id           int64
caregiver_id    float64
charttime        object
storetime        object
itemid            int64
value            object
valuenum        float64
valueuom         object
warning         float64
dtype: object

In [5]:
print("itemid 220615")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuCharDD_sCr = icuCharDD[icuCharDD["itemid"] == 220615]
icuCharDD_sCr=icuCharDD_sCr.dropna(subset=['valuenum'])

print("merge")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuCharDD_sCr_admin = icuCharDD_sCr.merge(admiDD, how="left", on=["subject_id","hadm_id"])
icuCharDD_sCr_admin=icuCharDD_sCr_admin.sort_values(by=['subject_id', 'hadm_id','charttime'])

print("end merge")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

#11min

itemid 220615
20230530_155355
merge
20230530_155355
end merge
20230530_160403


In [6]:
print("compute")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuChar_sCr_admin = icuCharDD_sCr_admin.compute()

print("end compute")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))
#7min

compute
20230530_160403
end compute
20230530_161024


In [7]:
print("to_datetime")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuChar_sCr_admin["charttime"] = pd.to_datetime(
    icuChar_sCr_admin["charttime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_admin["admittime"] = pd.to_datetime(
    icuChar_sCr_admin["admittime"], format="%Y-%m-%d %H:%M:%S"
)


icuChar_sCr_admin["dischtime"] = pd.to_datetime(
    icuChar_sCr_admin["dischtime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_admin["deathtime"] = pd.to_datetime(
    icuChar_sCr_admin["deathtime"], format="%Y-%m-%d %H:%M:%S"
)


print("end to_datetime")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

to_datetime
20230530_161024
end to_datetime
20230530_161025


In [8]:
icuChar_sCr_admin=icuChar_sCr_admin.dropna(subset=['valuenum'])

In [9]:
staticsCr=pd.DataFrame()

In [10]:
staticsCr.loc[0,'max']=np.max(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'min']=np.min(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'std']=np.std(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'average']=np.average(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'mean']=np.mean(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'median']=np.median(icuChar_sCr_admin['valuenum'])
staticsCr.loc[0,'p0.1']=np.percentile(icuChar_sCr_admin['valuenum'],0.1)
staticsCr.loc[0,'p10']=np.percentile(icuChar_sCr_admin['valuenum'],10)
staticsCr.loc[0,'p20']=np.percentile(icuChar_sCr_admin['valuenum'],20)
staticsCr.loc[0,'p25']=np.percentile(icuChar_sCr_admin['valuenum'],25)
staticsCr.loc[0,'p75']=np.percentile(icuChar_sCr_admin['valuenum'],75)
staticsCr.loc[0,'p80']=np.percentile(icuChar_sCr_admin['valuenum'],80)
staticsCr.loc[0,'p90']=np.percentile(icuChar_sCr_admin['valuenum'],90)
staticsCr.loc[0,'p99.9']=np.percentile(icuChar_sCr_admin['valuenum'],99.9)

In [11]:
staticsCr

,max,min,std,average,mean,median,p0.1,p10,p20,p25,p75,p80,p90,p99.9
0,999999.0,-0.1,7353.19044,55.631237,55.631237,1.0,0.2,0.5,0.7,0.7,1.8,2.1,3.2,13.8


In [12]:
icuChar_sCr_admin_clean=icuChar_sCr_admin[(staticsCr.iloc[0]['p0.1']<=icuChar_sCr_admin['valuenum'])&(icuChar_sCr_admin['valuenum']<=staticsCr.iloc[0]['p99.9'])]

In [13]:
staticsCr.loc[0,'max_clean']=np.max(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'min_clean']=np.min(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'std_clean']=np.std(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'average_clean']=np.average(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'mean_clean']=np.mean(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'median_clean']=np.median(icuChar_sCr_admin_clean['valuenum'])
staticsCr.loc[0,'p0.1_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],0.1)
staticsCr.loc[0,'p10_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],10)
staticsCr.loc[0,'p20_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],20)
staticsCr.loc[0,'p25_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],25)
staticsCr.loc[0,'p75_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],75)
staticsCr.loc[0,'p80_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],80)
staticsCr.loc[0,'p90_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],90)
staticsCr.loc[0,'p99.9_clean']=np.percentile(icuChar_sCr_admin_clean['valuenum'],99.9)

In [14]:
staticsCr

,max,min,std,average,mean,median,p0.1,p10,p20,p25,...,mean_clean,median_clean,p0.1_clean,p10_clean,p20_clean,p25_clean,p75_clean,p80_clean,p90_clean,p99.9_clean
0,999999.0,-0.1,7353.19044,55.631237,55.631237,1.0,0.2,0.5,0.7,0.7,...,1.544656,1.0,0.2,0.5,0.7,0.7,1.8,2.1,3.2,11.8


In [15]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
staticsCr.to_excel(saveTimeStr+'_mimiciv_icuChar_sCr_statistical.xlsx',index=False)

In [16]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
icuChar_sCr_admin_clean.to_csv(saveTimeStr+'_mimiciv_icuChar_sCr_clean.csv',index=False)

In [17]:
icuChar_sCr_admin_clean.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime',
       'storetime', 'itemid', 'value', 'valuenum', 'valueuom', 'warning',
       'admittime', 'dischtime', 'deathtime', 'race', 'hospital_expire_flag'],
      dtype='object')

In [18]:
icuChar_sCr_admin_clean["delta_in_char_time(hours)"] = (
    icuChar_sCr_admin_clean["charttime"] - icuChar_sCr_admin_clean["admittime"]
) / np.timedelta64(1, "h")

icuChar_sCr_admin_clean["delta_char_disc_time(hours)"] = (
    icuChar_sCr_admin_clean["charttime"] - icuChar_sCr_admin_clean["dischtime"]
) / np.timedelta64(1, "h")

icuChar_sCr_admin_clean["delta_char_death_time(hours)"] = (
    icuChar_sCr_admin_clean["charttime"] - icuChar_sCr_admin_clean["deathtime"]
) / np.timedelta64(1, "h")

icuChar_sCr_admin_clean['delta_time(hours)']=icuChar_sCr_admin_clean["delta_in_char_time(hours)"]

C:\Users\apple\AppData\Local\Temp\ipykernel_9960\36253638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icuChar_sCr_admin_clean["delta_in_char_time(hours)"] = (
C:\Users\apple\AppData\Local\Temp\ipykernel_9960\36253638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icuChar_sCr_admin_clean["delta_char_disc_time(hours)"] = (
C:\Users\apple\AppData\Local\Temp\ipykernel_9960\36253638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [19]:
icuChar_sCr_admin_clean_intime=icuChar_sCr_admin_clean[(0<icuChar_sCr_admin_clean["delta_in_char_time(hours)"])
                                            &(icuChar_sCr_admin_clean["delta_in_char_time(hours)"]<=168)
                                            &(icuChar_sCr_admin_clean["delta_char_disc_time(hours)"]<=0)]

icuCharteventssSCrCut=icuChar_sCr_admin_clean_intime.sort_values(by=['subject_id', 'hadm_id','charttime'])

In [20]:
icuChar_sCr_admin_clean_intime.shape

(286813, 20)

In [21]:
uni_hadm_id=pd.unique(icuChar_sCr_admin_clean_intime['hadm_id'])
print(len(uni_hadm_id))

61945


In [22]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
icuChar_sCr_admin_clean_intime.to_csv(saveTimeStr+'mimiciv_icuChar_sCr_clean_inTime.csv',index=False)

In [ ]:
#quick redo point
icuChar_sCr_admin_clean_intime=pd.read_csv('mimiciv_icuChar_sCr_clean_outTime.csv')


icuChar_sCr_admin_clean_intime["charttime"] = pd.to_datetime(
    icuChar_sCr_admin_clean_intime["charttime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_admin_clean_intime["admittime"] = pd.to_datetime(
    icuChar_sCr_admin_clean_intime["admittime"], format="%Y-%m-%d %H:%M:%S"
)


icuChar_sCr_admin_clean_intime["dischtime"] = pd.to_datetime(
    icuChar_sCr_admin_clean_intime["dischtime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_admin_clean_intime["deathtime"] = pd.to_datetime(
    icuChar_sCr_admin_clean_intime["deathtime"], format="%Y-%m-%d %H:%M:%S"
)




In [23]:
icuChar_sCr_admin_clean_intime.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime',
       'storetime', 'itemid', 'value', 'valuenum', 'valueuom', 'warning',
       'admittime', 'dischtime', 'deathtime', 'race', 'hospital_expire_flag',
       'delta_in_char_time(hours)', 'delta_char_disc_time(hours)',
       'delta_char_death_time(hours)', 'delta_time(hours)'],
      dtype='object')

In [24]:
icuChar_sCr_admin_clean_intime[:20]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,admittime,dischtime,deathtime,race,hospital_expire_flag,delta_in_char_time(hours),delta_char_disc_time(hours),delta_char_death_time(hours),delta_time(hours)
0,10000032,29079034,39553978,NaN,2180-07-23 21:45:00,2180-07-23 22:30:00,220615,0.5,0.5,mg/dL,0.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,WHITE,0,9.166667,-44.166667,NaN,9.166667
1,10001217,24597018,37067082,NaN,2157-11-21 03:16:00,2157-11-21 04:53:00,220615,0.4,0.4,mg/dL,0.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,WHITE,0,52.333333,-110.733333,NaN,52.333333
2,10001217,27703517,34592300,NaN,2157-12-20 01:45:00,2157-12-20 02:41:00,220615,0.5,0.5,mg/dL,0.0,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,WHITE,0,32.783333,-109.166667,NaN,32.783333
3,10001725,25563031,31205490,NaN,2110-04-11 18:02:00,2110-04-11 18:53:00,220615,0.8,0.8,mg/dL,0.0,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,WHITE,0,2.900000,-68.966667,NaN,2.900000
4,10001725,25563031,31205490,NaN,2110-04-12 02:59:00,2110-04-12 03:47:00,220615,0.8,0.8,mg/dL,0.0,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,WHITE,0,11.850000,-60.016667,NaN,11.850000
5,10001884,26184834,37510196,NaN,2131-01-11 06:31:00,2131-01-11 07:52:00,220615,1.1,1.1,mg/dL,0.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,BLACK/AFRICAN AMERICAN,1,81.866667,-214.733333,-214.733333,81.866667
6,10001884,26184834,37510196,NaN,2131-01-12 03:34:00,2131-01-12 05:47:00,220615,1,1.0,mg/dL,0.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,BLACK/AFRICAN AMERICAN,1,102.916667,-193.683333,-193.683333,102.916667
7,10001884,26184834,37510196,NaN,2131-01-13 04:29:00,2131-01-13 05:54:00,220615,1.2,1.2,mg/dL,1.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,BLACK/AFRICAN AMERICAN,1,127.833333,-168.766667,-168.766667,127.833333
8,10001884,26184834,37510196,NaN,2131-01-13 08:23:00,2131-01-13 10:33:00,220615,1.3,1.3,mg/dL,1.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,BLACK/AFRICAN AMERICAN,1,131.733333,-164.866667,-164.866667,131.733333
9,10001884,26184834,37510196,NaN,2131-01-13 10:35:00,2131-01-13 12:29:00,220615,1.2,1.2,mg/dL,1.0,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,BLACK/AFRICAN AMERICAN,1,133.933333,-162.666667,-162.666667,133.933333


In [25]:
def tKDIGO_kn(inputt):
    out = pd.DataFrame()

    out.loc[0, 'subject_id'] = inputt.iloc[0]['subject_id']
    out.loc[0, 'hadm_id'] = inputt.iloc[0]['hadm_id']
    out.loc[0, 'admittime'] = inputt.iloc[0]['admittime']

    akiHappenAt = inputt.iloc[len(inputt)-1]['delta_time(hours)']

    if len(inputt) < 2:
        out.loc[0, 'AKI_happened'] = np.nan
        out.loc[0, 'AKI_time'] = np.nan
        out.loc[0, 'KDIGO_Stage'] = np.nan
        out.loc[0, 'deltCre01'] = np.nan
        out.loc[0, 'deltCre02'] = np.nan
        out.loc[0, 'delta_time(hours)'] = np.nan
        return out
    # =================================================================================================
    tmp = inputt[(inputt['delta_time(hours)'] <= 48)]
    deltCre01 = 0
    chtime01 = inputt.iloc[0]['charttime']
    if len(tmp) > 1:
        #deltCre01 = tmp['valuenum'].max()-tmp['valuenum'].min()
        for ii in range(len(tmp)-1):
            v1 = tmp.iloc[0]['valuenum']
            v2 = tmp.iloc[ii+1]['valuenum']
            v3 = v2-v1
            if v3 >= 0.3:
                ftt = tmp.iloc[ii+1]['delta_time(hours)']
                if akiHappenAt > ftt:
                    akiHappenAt = ftt
            if deltCre01 < v3:
                deltCre01 = v3
                chtime01 = tmp.iloc[ii+1]['charttime']

    # =================================================================================================
    tmp = inputt[(inputt['delta_time(hours)'] <= 168)]
    deltCre02 = 0
    chtime02 = inputt.iloc[0]['charttime']
    if len(tmp) > 1:
        for ii in range(len(tmp)-1):
            v1 = tmp.iloc[0]['valuenum']
            v2 = tmp.iloc[ii+1]['valuenum']
            v3 = (v2-v1)/v1
            if v3 >= 1.5:
                ftt = tmp.iloc[ii+1]['delta_time(hours)']
                if akiHappenAt > ftt:
                    akiHappenAt = ftt
            if deltCre02 < v3:
                deltCre02 = v3
                chtime02 = tmp.iloc[ii+1]['charttime']

    # =================================================================================================
    if deltCre01 >= 0.3 or deltCre02 >= 1.5:
        out.loc[0, 'AKI_happened'] = 1
        hapnTime = (chtime02 - chtime01) / np.timedelta64(1, 'h')
        if hapnTime >= 0:
            out.loc[0, 'AKI_time'] = chtime02
        else:
            out.loc[0, 'AKI_time'] = chtime01

        if (deltCre01 >= 4) or (3 <= deltCre02):
            out.loc[0, 'KDIGO_Stage'] = 3
        elif (2 <= deltCre02) and (deltCre02 < 3):
            out.loc[0, 'KDIGO_Stage'] = 2
        elif ((1.5 <= deltCre02) and (deltCre02 < 2)) or (0.3 <= deltCre01):
            out.loc[0, 'KDIGO_Stage'] = 1
        else:
            out.loc[0, 'KDIGO_Stage'] = -100

    else:
        out.loc[0, 'AKI_happened'] = 0
        out.loc[0, 'AKI_time'] = np.nan
        out.loc[0, 'KDIGO_Stage'] = 0

    # =================================================================================================
    out.loc[0, 'deltCre01'] = deltCre01
    out.loc[0, 'deltCre02'] = deltCre02
    out.loc[0, 'delta_time(hours)'] = akiHappenAt

    return out




In [26]:
mimiciv_icu_aki=icuChar_sCr_admin_clean_intime.groupby(['subject_id','hadm_id']).apply(tKDIGO_kn).reset_index(drop=True)
#7min

In [27]:
mimiciv_icu_aki=mimiciv_icu_aki.dropna(subset=['AKI_happened'])
#mimiciv_icu_aki.index=range(len(mimiciv_icu_aki))
mimiciv_icu_aki=mimiciv_icu_aki.reset_index()

In [28]:
mimiciv_icu_aki[:20]

,index,subject_id,hadm_id,admittime,AKI_happened,AKI_time,KDIGO_Stage,deltCre01,deltCre02,delta_time(hours)
0,3,10001725.0,25563031.0,2110-04-11 15:08:00,0.0,NaT,0.0,0.0,0.000000,11.850000
1,4,10001884.0,26184834.0,2131-01-07 20:39:00,0.0,NaT,0.0,0.0,0.181818,162.850000
2,5,10002013.0,23581541.0,2160-05-18 07:45:00,0.0,NaT,0.0,0.0,0.000000,20.250000
3,7,10002155.0,23822395.0,2129-08-04 12:44:00,0.0,NaT,0.0,0.3,0.333333,137.000000
4,8,10002155.0,28994087.0,2130-09-23 21:59:00,0.0,NaT,0.0,0.0,0.000000,74.200000
5,9,10002348.0,22725460.0,2112-11-30 22:22:00,0.0,NaT,0.0,0.0,0.125000,151.800000
6,10,10002428.0,20321825.0,2156-04-30 20:35:00,0.0,NaT,0.0,0.1,0.333333,30.766667
7,11,10002428.0,23473524.0,2156-05-11 14:49:00,0.0,NaT,0.0,0.0,0.000000,157.333333
8,12,10002428.0,28662225.0,2156-04-12 14:16:00,1.0,2156-04-16 02:48:00,1.0,0.4,0.625000,38.366667
9,13,10002430.0,26295318.0,2129-06-13 00:00:00,0.0,NaT,0.0,0.1,0.090909,62.400000


In [29]:
pp=pd.unique(mimiciv_icu_aki['subject_id'])
print(len(pp))

40747


In [30]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
mimiciv_icu_aki.to_csv(saveTimeStr+'mimiciv_icu_aki.csv',index=False)

In [31]:
mimiciv_icu_aki.columns

Index(['index', 'subject_id', 'hadm_id', 'admittime', 'AKI_happened',
       'AKI_time', 'KDIGO_Stage', 'deltCre01', 'deltCre02',
       'delta_time(hours)'],
      dtype='object')

In [32]:
mimiciv_icu_aki[:5]

,index,subject_id,hadm_id,admittime,AKI_happened,AKI_time,KDIGO_Stage,deltCre01,deltCre02,delta_time(hours)
0,3,10001725.0,25563031.0,2110-04-11 15:08:00,0.0,NaT,0.0,0.0,0.000000,11.85
1,4,10001884.0,26184834.0,2131-01-07 20:39:00,0.0,NaT,0.0,0.0,0.181818,162.85
2,5,10002013.0,23581541.0,2160-05-18 07:45:00,0.0,NaT,0.0,0.0,0.000000,20.25
3,7,10002155.0,23822395.0,2129-08-04 12:44:00,0.0,NaT,0.0,0.3,0.333333,137.00
4,8,10002155.0,28994087.0,2130-09-23 21:59:00,0.0,NaT,0.0,0.0,0.000000,74.20


In [33]:
def tSelectFrist(inDf):
    inDf=inDf.sort_values(by=['admittime'])
    return inDf.iloc[0]

In [34]:
mimiciv_icu_aki_frist=mimiciv_icu_aki.groupby(['subject_id']).apply(tSelectFrist).reset_index(drop=True)
mimiciv_icu_aki_frist=mimiciv_icu_aki_frist.sort_values(by=['subject_id', 'hadm_id'])

mimiciv_icu_aki_frist=mimiciv_icu_aki_frist[['subject_id', 'hadm_id', 'AKI_happened', 'AKI_time',
       'KDIGO_Stage', 'delta_time(hours)']]

mimiciv_icu_aki_frist=mimiciv_icu_aki_frist.reset_index()

In [35]:
mimiciv_icu_aki_frist

,index,subject_id,hadm_id,AKI_happened,AKI_time,KDIGO_Stage,delta_time(hours)
0,0,10001725.0,25563031.0,0.0,NaT,0.0,11.850000
1,1,10001884.0,26184834.0,0.0,NaT,0.0,162.850000
2,2,10002013.0,23581541.0,0.0,NaT,0.0,20.250000
3,3,10002155.0,23822395.0,0.0,NaT,0.0,137.000000
4,4,10002348.0,22725460.0,0.0,NaT,0.0,151.800000
...,...,...,...,...,...,...,...
40742,40742,19999442.0,26785317.0,0.0,NaT,0.0,162.150000
40743,40743,19999625.0,25304202.0,0.0,NaT,0.0,20.816667
40744,40744,19999828.0,25744818.0,0.0,NaT,0.0,37.233333
40745,40745,19999840.0,21033226.0,0.0,NaT,0.0,167.433333


In [36]:
mimiciv_icu_aki_frist.columns

Index(['index', 'subject_id', 'hadm_id', 'AKI_happened', 'AKI_time',
       'KDIGO_Stage', 'delta_time(hours)'],
      dtype='object')

In [37]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
mimiciv_icu_aki_frist.to_csv(saveTimeStr+'mimiciv_icu_aki_frist.csv',index=False)

In [38]:
pp=pd.unique(mimiciv_icu_aki_frist['subject_id'])
print(len(pp))

40747


In [39]:
pp=pd.unique(mimiciv_icu_aki_frist['hadm_id'])
print(len(pp))

40747
